## **Data Wrangling**

Perform the following operations using Python on any open source dataset (e.g., data.csv)
1. Import all the required Python Libraries.
2. Locate an open source data from the web (e.g., https://www.kaggle.com). Provide a clear
 description of the data and its source (i.e., URL of the web site).
3. Load the Dataset into pandas dataframe.
4. Data Preprocessing: check for missing values in the data using pandas isnull(), describe()
function to get some initial statistics. Provide variable descriptions. Types of variables etc.
Check the dimensions of the data frame.
5. Data Formatting and Data Normalization: Summarize the types of variables by checking
the data types (i.e., character, numeric, integer, factor, and logical) of the variables in the
data set. If variables are not in the correct data type, apply proper type conversions.
6. Turn categorical variables into quantitative variables in Python.


In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

job_data = pd.read_csv("./datasets/DS_jobs.csv")
print(job_data.head())

   index          Job Title               Salary Estimate  \
0      0  Sr Data Scientist  $137K-$171K (Glassdoor est.)   
1      1     Data Scientist  $137K-$171K (Glassdoor est.)   
2      2     Data Scientist  $137K-$171K (Glassdoor est.)   
3      3     Data Scientist  $137K-$171K (Glassdoor est.)   
4      4     Data Scientist  $137K-$171K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Description\n\nThe Senior Data Scientist is re...     3.1   
1  Secure our Nation, Ignite your Future\n\nJoin ...     4.2   
2  Overview\n\n\nAnalysis Group is one of the lar...     3.8   
3  JOB DESCRIPTION:\n\nDo you have a passion for ...     3.5   
4  Data Scientist\nAffinity Solutions / Marketing...     2.9   

              Company Name       Location            Headquarters  \
0         Healthfirst\n3.1   New York, NY            New York, NY   
1             ManTech\n4.2  Chantilly, VA             Herndon, VA   
2      Analysis Group\n3.8     Boston, MA

In [15]:
print(job_data.describe())

            index      Rating      Founded
count  672.000000  672.000000   672.000000
mean   335.500000    3.518601  1635.529762
std    194.133974    1.410329   756.746640
min      0.000000   -1.000000    -1.000000
25%    167.750000    3.300000  1917.750000
50%    335.500000    3.800000  1995.000000
75%    503.250000    4.300000  2009.000000
max    671.000000    5.000000  2019.000000


## 2. Check for missing values in the data

In [16]:

cols_with_nan = job_data.columns[job_data.isnull().any()].tolist()


print(f"No of columns with Nan: {len(cols_with_nan)}" if cols_with_nan else "No missing values")
print(f"\nColumns{" ":<12}Datatypes\n"+'*'*28)
print(job_data.dtypes)
print('*'*28+'\n')
print(f"Dimension of Job dataset is:- {job_data.shape}")

No missing values

Columns            Datatypes
****************************
index                  int64
Job Title                str
Salary Estimate          str
Job Description          str
Rating               float64
Company Name             str
Location                 str
Headquarters             str
Size                     str
Founded                int64
Type of ownership        str
Industry                 str
Sector                   str
Revenue                  str
Competitors              str
dtype: object
****************************

Dimension of Job dataset is:- (672, 15)


## 2. Salary and Location Parsing

In [17]:
# Extracting min-max Salary Estimate and converting them to numeric
salary_text = job_data['Salary Estimate'].str.replace('K', '').str.replace('$', '')
job_data[['min_salary', 'max_salary']] = salary_text.str.extract(r'(\d+)-(\d+)')
job_data['min_salary'] = pd.to_numeric(job_data['min_salary'])
job_data['max_salary'] = pd.to_numeric(job_data['max_salary'])

#
# job_data[['city', 'state']] = job_data['Location'].str.extract(r'^(.*),\s(\w{2})$')
job_data[['city', 'state']] = job_data['Location'].str.extract(r'^(.*?),\s*([A-Z]{2})$')

print(f"Missing Cities: {job_data['city'].isnull().sum()}")
print(f"Missing States: {job_data['state'].isnull().sum()}")


Missing Cities: 23
Missing States: 23


## 2.2 Clean Geographic Data

In [18]:
job_data.dropna(subset=['city', 'state'], inplace=True)

# Reset index after dropping rows
job_data.reset_index(drop=True, inplace=True)

print(f"Remaining rows after cleaning: {job_data.shape[0]}")

Remaining rows after cleaning: 649


## 3. Revenue Normalization

In [19]:
def map_revenue(rev_str):
    rev_str = str(rev_str).lower()
    if 'billion' in rev_str:
        if '1 to 2' in rev_str: return '1B'
        if '2 to 5' in rev_str: return '5B'
        if '5 to 10' in rev_str: return '10B'
        return '50B'
    elif 'million' in rev_str:
        if '100 to 500' in rev_str: return '500M'
        if '50 to 100' in rev_str: return '100M'
        if '25 to 50' in rev_str: return '50M'
        if '10 to 25' in rev_str: return '25M'
        return '10M'
    return 'Unknown'

job_data['Revenue_Mapped'] = job_data['Revenue'].apply(map_revenue)

## 4. Skill Extraction (One-Hot Encoding)

In [20]:
skills_list = ['python', 'excel', 'sql', 'aws', 'spark', 'tableau']

for skill in skills_list:
    job_data[skill] = job_data['Job Description'].apply(lambda x: 1 if skill in x.lower() else 0)

print(job_data[skills_list].head())

   python  excel  sql  aws  spark  tableau
0       0      0    0    1      0        0
1       0      0    1    0      0        0
2       1      1    0    1      0        0
3       1      1    1    1      0        0
4       1      1    1    0      0        0


## 5. Categorical Encoding

In [21]:
le = LabelEncoder()
cols_to_encode = ['Sector', 'Type of ownership', 'Industry']

for col in cols_to_encode:
    job_data[col] = job_data[col].replace('-1', 'Unknown').fillna('Unknown')
    job_data[f'{col}_label'] = le.fit_transform(job_data[col])

print(job_data[['Sector', 'Sector_label']].head())

              Sector  Sector_label
0          Insurance            12
1  Business Services             4
2  Business Services             4
3      Manufacturing            13
4  Business Services             4


In [24]:
# 1. Check final dimensions and missing values
print("### 1. DATASET DIMENSIONS & NULLS ###")
print(f"Final Shape: {job_data.shape}")
print(job_data[['city', 'state', 'min_salary', 'max_salary']].isnull().sum())
print("-" * 30)

# 2. Display the newly created Quantitative/Encoded columns
print("### 2. QUANTITATIVE & ENCODED COLUMNS ###")
cols_to_show = ['Sector_label', 'Industry_label', 'python', 'sql', 'aws', 'Revenue_Mapped']
print(job_data[cols_to_show].head())
print("-" * 30)

# 3. Summarize the Revenue Mapping
print("### 3. REVENUE DISTRIBUTION (Mapped) ###")
print(job_data['Revenue_Mapped'].value_counts())
print("-" * 30)

# 4. Final Data Types (Formatting Check)
print("### 4. DATA TYPES SUMMARY ###")
print(job_data.dtypes[['min_salary', 'max_salary', 'Sector_label', 'python']])

# 5. Save the cleaned data to a new CSV (Optional but recommended for practicals)
# job_data.to_csv("DS_jobs_cleaned.csv", index=False)

### 1. DATASET DIMENSIONS & NULLS ###
Final Shape: (649, 29)
city          0
state         0
min_salary    0
max_salary    0
dtype: int64
------------------------------
### 2. QUANTITATIVE & ENCODED COLUMNS ###
   Sector_label  Industry_label  python  sql  aws Revenue_Mapped
0            12              30       0    0    1        Unknown
1             4              42       0    1    0            50B
2             4              11       1    0    1            10M
3            13              15       1    1    1            10M
4             4               1       1    1    0        Unknown
------------------------------
### 3. REVENUE DISTRIBUTION (Mapped) ###
Revenue_Mapped
10M        253
Unknown    229
50B        167
Name: count, dtype: int64
------------------------------
### 4. DATA TYPES SUMMARY ###
min_salary      int64
max_salary      int64
Sector_label    int64
python          int64
dtype: object
